In [1]:
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
%matplotlib inline

def create_spectrogram(audio_file, image_file):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

    y, sr = librosa.load(audio_file)
    ms = librosa.feature.melspectrogram(y = y, sr = sr)
    log_ms = librosa.power_to_db(ms, ref=np.max)
    librosa.display.specshow(log_ms, sr=sr)

    fig.savefig(image_file)
    plt.close(fig)
    
def create_pngs_from_wavs(input_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)

    dir = os.listdir(input_path)

    for i, file in enumerate(dir):
        if '.txt' in file: continue
        input_file = os.path.join(input_path, file)
        output_file = os.path.join(output_path, file.replace('.wav', '.png'))
        create_spectrogram(input_file, output_file)

In [2]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/cel', 'Spectrograms/cel')

In [3]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/cla', 'Spectrograms/cla')

In [4]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/flu', 'Spectrograms/flu')

In [5]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/gac', 'Spectrograms/gac')

In [6]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/gel', 'Spectrograms/gel')

In [7]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/org', 'Spectrograms/org')

In [8]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/pia', 'Spectrograms/pia')

In [9]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/sax', 'Spectrograms/sax')

In [10]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/tru', 'Spectrograms/tru')

In [11]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/vio', 'Spectrograms/vio')

In [12]:
create_pngs_from_wavs('Dataset/IRMAS_Training_Data/voi', 'Spectrograms/voi')

In [2]:
# from keras.preprocessing import image
import keras.utils as image
import os
import matplotlib.pyplot as plt
from random import randint

NUM_INSTRUMENTS = 11

instruments = ["cel", "cla", "flu", "gac", "gel", "org", "pia", "sax", "tru", "vio", "voi"]

inst_dict = {}
for i in range(len(instruments)): 
    inst_dict[instruments[i]] = i

def load_images_from_path(path, x_array, y_array, index, label = None, path_label = None):
    images = []
    labels = []
    temp_label = [0 for _ in range(NUM_INSTRUMENTS)]
    if not path_label:
        temp_label[label] = 1     
        
        
    for file in os.listdir(path):
        x_array[index] = image.img_to_array(image.load_img(os.path.join(path, file), target_size=(224, 224, 3)))
        
        # images.append(image.img_to_array(image.load_img(os.path.join(path, file), target_size=(224, 224, 3))))
        if path_label:
            temp_label = [False for _ in range(NUM_INSTRUMENTS)]
            with open("{}/{}.txt".format(path_label, file[:-4]), "r") as f:
                for inst in f:
                    inst = inst.replace('\n', '').replace('\t', '')
                    temp_label[inst_dict[inst]] = 1
        # labels.append((temp_label))
        y_array[index] = temp_label
        index += 1
        
        
    # print(len(images), len(os.listdir(path)), len(images[0]))
    # return images, labels
    return index

def show_images(images):
    fig, axes = plt.subplots(1, 8, figsize=(20, 20), subplot_kw={'xticks': [], 'yticks': []})

    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i] / 255)
        

In [3]:
from tensorflow.keras.utils import to_categorical
len_train = 0
for i in range(len(instruments)):
    len_train += len(os.listdir('Spectrograms/{}'.format(instruments[i])))
print(len_train)

x = np.empty(shape = (len_train, 224, 224, 3))
y = np.empty(shape = (len_train, NUM_INSTRUMENTS))
index = 0
for i in range(len(instruments)):
    index = load_images_from_path(path = 'Spectrograms/{}'.format(instruments[i]), label = i, x_array = x, y_array = y, index = index)
    # images, labels = load_images_from_path(path = 'Spectrograms/{}'.format(instruments[i]), label = i)
    # show_images(images)
    
    # np.append(x, images)
    # np.append(y, labels)

y_enc = to_categorical(y)
print(len(x))
print(len(y))

7210
7210
7210


In [ ]:
create_pngs_from_wavs('Dataset/IRMAS_Validation_Data', 'Spectrograms/TEST')

In [4]:
print(x[::100])
print(y[::100])

[[[[  0.   0.   2.]
   [  0.   0.   3.]
   [  0.   0.   3.]
   ...
   [ 26.  15.  64.]
   [ 68.  15. 113.]
   [ 68.  15. 113.]]

  [[  0.   0.   3.]
   [  0.   0.   4.]
   [  0.   0.   4.]
   ...
   [ 28.  16.  68.]
   [ 73.  16. 120.]
   [ 73.  16. 120.]]

  [[  2.   2.  17.]
   [  3.   3.  18.]
   [  0.   0.   4.]
   ...
   [ 29.  17.  71.]
   [ 74.  16. 121.]
   [ 74.  16. 121.]]

  ...

  [[161.  48. 117.]
   [171.  51. 124.]
   [153.  45. 128.]
   ...
   [140.  41. 129.]
   [173.  52. 124.]
   [173.  52. 124.]]

  [[156.  46. 118.]
   [166.  49. 125.]
   [156.  46. 127.]
   ...
   [196.  60. 117.]
   [189.  57. 119.]
   [189.  57. 119.]]

  [[156.  46. 118.]
   [166.  49. 125.]
   [156.  46. 127.]
   ...
   [196.  60. 117.]
   [189.  57. 119.]
   [189.  57. 119.]]]


 [[[  0.   0.   2.]
   [  0.   0.   3.]
   [  0.   0.   3.]
   ...
   [ 31.  16.  73.]
   [ 74.  17. 116.]
   [ 74.  17. 116.]]

  [[  0.   0.   3.]
   [  0.   0.   4.]
   [  0.   0.   4.]
   ...
   [ 33.  17.  78.]
 

In [5]:
x_test = np.empty(shape = (len(os.listdir('Spectrograms/TEST')), 224, 224, 3))
y_test = np.empty(shape = (len(os.listdir('Spectrograms/TEST')), NUM_INSTRUMENTS))
y_test_enc = to_categorical(y_test)
load_images_from_path(path = 'Spectrograms/TEST', path_label = 'Dataset/IRMAS_Validation_Data', x_array = x_test, y_array = y_test, index = 0)

2874

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense
import keras.backend
from keras.losses import losses_utils
import keras.losses


# model = Sequential()

# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
# model.add(MaxPooling2D(2, 2))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(2, 2))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(2, 2))

# # model.add(Conv2D(128, (3, 3), activation='relu'))
# # model.add(MaxPooling2D(2, 2))

# model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(MaxPooling2D(2, 2))

# model.add(Flatten())

# model.add(Dense(1024, activation='relu'))
# model.add(Dense(512, activation = 'relu'))

# model.add(Dense(11, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(11, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 186624)            0         
                                                                 
 dense (Dense)               (None, 128)               2

In [7]:
print(np.shape(x))
print(np.shape(y))
print(np.shape(x_test))
print(np.shape(y_test))

(7210, 224, 224, 3)
(7210, 11)
(2874, 224, 224, 3)
(2874, 11)


In [8]:
print(len(x), len(y), len(x_test), len(y_test))
hist = model.fit(x, y, validation_data=(x_test, y_test), batch_size=32, epochs=20, shuffle = True, use_multiprocessing= True, workers = 4)

7210 7210 2874 2874
Epoch 1/20
226/226 [==============================] - 280s 1s/step - loss: 13.1073 - accuracy: 0.1773 - val_loss: 0.4419 - val_accuracy: 0.1343
Epoch 2/20
226/226 [==============================] - 265s 1s/step - loss: 0.2717 - accuracy: 0.2731 - val_loss: 0.4565 - val_accuracy: 0.1385
Epoch 3/20
226/226 [==============================] - 265s 1s/step - loss: 0.2465 - accuracy: 0.3603 - val_loss: 0.4885 - val_accuracy: 0.1447
Epoch 4/20
226/226 [==============================] - 262s 1s/step - loss: 0.2229 - accuracy: 0.4284 - val_loss: 0.4983 - val_accuracy: 0.2008
Epoch 5/20
226/226 [==============================] - 258s 1s/step - loss: 0.2062 - accuracy: 0.4899 - val_loss: 0.5930 - val_accuracy: 0.1340
Epoch 6/20
226/226 [==============================] - 257s 1s/step - loss: 0.1862 - accuracy: 0.5483 - val_loss: 0.6188 - val_accuracy: 0.1795
Epoch 7/20
226/226 [==============================] - 258s 1s/step - loss: 0.1732 - accuracy: 0.6000 - val_loss: 0.6954 -

In [9]:
model.save('Model_final')

INFO:tensorflow:Assets written to: Model_final\assets


INFO:tensorflow:Assets written to: Model_final\assets


In [10]:
from keras.models import load_model
new_model = load_model('Model')

In [23]:
import numpy as np
np.set_printoptions(precision=1)
np.set_printoptions(suppress = True)
acc = [[0, 0], [0, 0]]
for i in range(len(x_test)):
    for x, y in zip( model.predict(np.array([x_test[i]]))[0], y_test[i]):
        if x > 0.5 and y == 1: acc[1][1] += 1
        elif y == 1: acc[1][0] += 1
        elif x > 0.5 and y == 0: acc[0][1] += 1
        else: acc[0][0] += 1
print(acc)

1/1 [==============================] - 0s 64ms/step
[[25252, 1445], [4187, 730]]


In [25]:
print(len(x_test) * 11)

31614
